# Edge AI Anomaly Detection

## Overview

This document contains the code for our EclipseCON 2022 Talk: "_How to Train Your Dragon and Its Friends: AI on the Edge with Eclipse Kura&trade;_"

In this notebook we'll cover:
- **Data collection**: how to retrieve training data from the field leveragin Eclipse Kura&trade; and Eclipse Kapua&trade;
- **Data exploration**: how to explore our training data and manipulate them to make them suitable for training
- **Model training**: how we can create a simple Autoencoder in Keras and how to train it
- **Model performance evaluation**: how can we ensure the model was trained correctly
- **Model conversion**: how we can convert the model to make it suitable for running on Eclipse Kura&trade; and Nvidia Triton&trade;
- **Model deployment**: how can we deploy our model on our final device and use it for real-time anomaly detection

## Data collection

[WIP]

## Data Exploration

We can now work on our `.csv` file downloaded from Kapua. For demonstration purposes a copy is provided within this repository.

In [4]:
!ls *.csv

train-data-raw.csv
